# Exploring Eco topics with Python | TEST NOTEBOOK - Exploring Deforestation (with Rioxarray)
---

## A.

In [ ]:
import glob
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt

In [ ]:
path = "DeforestationExp_DATA"
extension = ".tif"
rasters = glob.glob(path + "/*" + extension)

for i in rasters:
    globals()[f'{i[len(path)+1 : -len(extension)]}'] = rxr.open_rasterio(i, masked=True).squeeze()

## B.

In [ ]:
ROI_1984_redband3.shape

In [ ]:
ROI_1984_redband3.dtype

In [ ]:
ROI_1984_redband3.plot(cmap="Reds_r", aspect=1.3, size=4)

In [ ]:
ROI_1984_redband3.rio.bounds()

In [ ]:
ROI_1984_redband3.values[:5, :5]

In [ ]:
plt.matshow(ROI_1984_redband3.values[:5, :5], cmap="Reds_r")

In [ ]:
array = ROI_1984_redband3.values[:5, :5]
fig, ax = plt.subplots(figsize=(5,5))    
ax.matshow(array, cmap="Reds_r")
for y in range(array.shape[0]):
    for x in range(array.shape[1]):
        plt.text(x,y,array[y,x], bbox=dict(facecolor='beige'), horizontalalignment="center")

## C.

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14, 5))
ROI_2022_redband4.plot(cmap="Reds_r", ax=ax[0], robust=True)
ROI_2022_nirband5.plot(cmap="Blues_r", ax=ax[1], robust=True)
plt.show()

## D.

In [ ]:
NDVI_1984 = (ROI_1984_nirband4 - ROI_1984_redband3) / (ROI_1984_nirband4 + ROI_1984_redband3)

In [ ]:
NDVI_1984.values[:2, :2]

In [ ]:
NDVI_2022 = (ROI_2022_nirband5 - ROI_2022_redband4) / (ROI_2022_nirband5 + ROI_2022_redband4)

In [ ]:
NDVI_2022.values[:2, :2]

## E.

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14, 5))
NDVI_1984.plot(cmap="terrain_r", ax=ax[0], vmin=-0.15, vmax=0.75)
NDVI_2022.plot(cmap="terrain_r", ax=ax[1], vmin=-0.15, vmax=0.75)
plt.show()

In [ ]:
NDVI_1984.plot.hist(color="royalblue")
NDVI_2022.plot.hist(color="m", alpha=0.5);

## F.

In [ ]:
type(NDVI_2022)

In [ ]:
LCLU = NDVI_2022.copy()

In [ ]:
LCLU_1 = xr.where(LCLU < 0.15, 100, LCLU)
LCLU_2 = xr.where(LCLU_1 < 0.25, 200, LCLU_1)
LCLU_3 = xr.where(LCLU_2 < 0.4, 300, LCLU_2)
LCLU_4 = xr.where(LCLU_3 <= 1, 400, LCLU_3)

In [ ]:
LCLU_4.plot()

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
LCLU_cmap = ListedColormap(["sandybrown", "gold", "yellowgreen", "forestgreen"])

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
LCLU_4.plot(cmap=LCLU_cmap)
colorbar=ax.collections[0].colorbar
colorbar.set_ticks([100,200,300,400])
colorbar.set_ticklabels(["Rock/Bare Soil/Water/Concrete", "Sparse/Senescing Veg1", "Sparse/Senescing Veg2", "Denser Veg"])
plt.show()

## G.

In [ ]:
from pyproj import Transformer, CRS

In [ ]:
NDVI_2022.rio.crs

In [ ]:
CRS.from_epsg(32620)

In [ ]:
epsg4326_long = -61.00
epsg4326_lat = -13.55

In [ ]:
transformer_4326to32620 = Transformer.from_crs("EPSG:4326", NDVI_2022.rio.crs, always_xy=True)

In [ ]:
epsg32620_POI_x, epsg32620_POI_y = transformer_4326to32620.transform(epsg4326_long, epsg4326_lat)

In [ ]:
epsg32620_POI_x, epsg32620_POI_y

In [ ]:
NDVI_2022.sel(x=epsg32620_POI_x, y=epsg32620_POI_y, method="nearest").values

In [ ]:
fig, ax = plt.subplots(figsize=(11,8))
LCLU_4.plot(cmap=LCLU_cmap)
colorbar=ax.collections[0].colorbar
colorbar.set_ticks([100,200,300,400])
colorbar.set_ticklabels(["Rock/Bare Soil/Water/Concrete", "Sparse/Senescing Veg1", "Sparse/Senescing Veg2", "Denser Veg"])
ax.annotate("New deforestation for pasture", xy=(epsg32620_POI_x, epsg32620_POI_y), arrowprops=dict(facecolor="Red"), textcoords="axes fraction", xytext = (0.5, 0.15), fontsize=12)   
plt.show()

---
Copyright © 2023 Rho Zeta AI Ltd. All rights reserved.